In [1]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def create_stack(ystack, axis, savetodir, downsample_factor, swapxz = False, scalebar_microns = 100):
    '''
    Creates a stack of images along the specified axis, using a source y-stack of images.
    Args:
        ystack: A list of images (np array) to stack.
        axis: The axis along which to stack the images. Can be 'x', 'y', or 'z'.
        downsample_factor: Factor by which to downsample the image in all three dimensions. Default is 1 (no downsampling).
        swapxz: Whether to swap the x and z axes. Default is False.
    '''
    # Downsample every image in the source stack
    ystack = [cv2.resize(img, (img.shape[1]//downsample_factor, img.shape[0]//downsample_factor)) for img in ystack]

    # Delete every downsample_factor image in the source stack
    ystack = [img for i, img in enumerate(ystack) if i % downsample_factor == 0]

    if swapxz:
        # Swap x and z axes
        ystack = [np.swapaxes(img, 0, 1).astype(np.uint8) for img in ystack]

    if axis == 'x':
        print('Creating x stack...')
        stack = [np.zeros((len(ystack), ystack[0].shape[0], 3), dtype=np.uint8) for _ in range(ystack[0].shape[1])]
        for x in tqdm(range(ystack[0].shape[1])):
            edges = [img[:, x:x+1, :].reshape(1, img.shape[0], 3) for img in ystack]
            for i, edge in enumerate(edges):
                stack[x][i:i+1, :, :] = edge

    elif axis == 'z':
        print('Creating z stack...')
        stack = [np.zeros((len(ystack), ystack[0].shape[1], 3), dtype=np.uint8) for _ in range(ystack[0].shape[0])]
        for z in tqdm(range(ystack[0].shape[0])):
            edges = [img[z:z+1, :, :].reshape(1, img.shape[1], 3) for img in ystack]
            for i, edge in enumerate(edges):
                stack[z][i:i+1, :, :] = edge

    elif axis == 'y':
        print('Creating y stack...')
        stack = []
        for i, img in tqdm(enumerate(ystack), total = len(ystack)):
            # Dim the scale bar background
            img[-90:-15, -160:-10, :] = img[-90:-15, -160:-10, :] * 0.5
            # Add a scale bar to the bottom right corner of the image
            img[-80:-70, -135:-35, :] = 255
            # Annotate scale bar with length
            img = img.copy()
            cv2.putText(img, f'{scalebar_microns} um', (img.shape[1]-150, img.shape[0]-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            stack.append(img)
    else:
        raise Exception('axis must be one of x, y, or z')

    os.makedirs(os.path.join(savetodir, axis), exist_ok = True)
    for i, img in enumerate(stack):
        cv2.imwrite(os.path.join(savetodir, axis, f'{i}.png'), img)

In [2]:
for ID in ['(OTLS) c002', '(OTLS) c040', '(OTLS) c049']:
    TYPE = 'OTLS'

    for mode in ['raw', 'heatmap']:
        SOURCE_DIR = f'data/{ID}/{mode}/orig'
        DEST_DIR = f'data_hi_res/{ID}/{mode}'
        DOWNSAMPLE_FACTOR = 1
        if TYPE == 'CT':
            SCALEBAR_MICRONS = 400
            SWAP_XZ = False
        elif TYPE == 'OTLS':
            SCALEBAR_MICRONS = 100
            SWAP_XZ = True
        else:
            raise Exception(f'Invalid type: {TYPE} not in ["CT", "OTLS"]')

        # Load images
        imagepaths = sorted(os.listdir(SOURCE_DIR), key = lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort paths by index
        images = [cv2.imread(os.path.join(SOURCE_DIR, f)) for f in imagepaths]
        print(f'Creating xyz stacks from {len(images)} images in {SOURCE_DIR}...')

        # Create stacks
        create_stack(images, 'x', DEST_DIR, DOWNSAMPLE_FACTOR, SWAP_XZ)
        create_stack(images, 'z', DEST_DIR, DOWNSAMPLE_FACTOR, SWAP_XZ)
        create_stack(images, 'y', DEST_DIR, DOWNSAMPLE_FACTOR, SWAP_XZ, SCALEBAR_MICRONS)
        
        # Print dimensions
        print(f'Width: {len(os.listdir(os.path.join(DEST_DIR, "x")))}')
        print(f'Depth: {len(os.listdir(os.path.join(DEST_DIR, "z")))}')
        print(f'Height: {len(os.listdir(os.path.join(DEST_DIR, "y")))}')

Creating xyz stacks from 300 images in data/(OTLS) c002/raw/orig...
Creating x stack...


100%|██████████| 7065/7065 [00:25<00:00, 273.67it/s]


Creating z stack...


100%|██████████| 523/523 [00:40<00:00, 12.97it/s]
